In [1]:
# from google.colab import drive
# drive.mount('/content/drive')


In [3]:
# !pip install -q tensorflow-recommenders
# !pip install -q --upgrade tensorflow-datasets
# # !pip install -q scann

ERROR: scann 1.2.6 has requirement tensorflow~=2.8.0, but you'll have tensorflow 2.9.1 which is incompatible.
ERROR: tensorflow-recommenders 0.7.0 has requirement tensorflow>=2.9.0, but you'll have tensorflow 2.8.2 which is incompatible.


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
from typing import Dict, Text
import random
from collections import Counter

In [6]:
path = "./"
article_df = pd.read_csv(f"{path}/hmdata/articles.csv.zip")
customer_df = pd.read_csv(f"{path}/hmdata/customers.csv.zip")
train0 = pd.read_csv(f'{path}/hmdata/transactions_train.csv.zip')
transaction_df = train0

In [7]:
transaction_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [8]:
article_df["product_group_name"].unique()

array(['Garment Upper body', 'Underwear', 'Socks & Tights',
       'Garment Lower body', 'Accessories', 'Items', 'Nightwear',
       'Unknown', 'Underwear/nightwear', 'Shoes', 'Swimwear',
       'Garment Full body', 'Cosmetic', 'Interior textile', 'Bags',
       'Furniture', 'Garment and Shoe care', 'Fun', 'Stationery'],
      dtype=object)

# **Simple Feature Processing**

Having looked at some initial analysis, we next perform simple feature processing for customer and article dataframes.

In [9]:
def create_age_interval(x):
    if x <= 25:
        return [16, 25]
    elif x <= 35:
        return [26, 35]
    elif x <= 45:
        return [36, 45]
    elif x <= 55:
        return [46, 55]
    elif x <= 65:
        return [56, 65]
    else:
        return [66, 99]

customer_df["FN"].fillna(0, inplace=True)
customer_df["Active"].fillna(0, inplace=True)

# Set unknown the club member status & news frequency
customer_df["club_member_status"].fillna("UNKNOWN", inplace=True)

customer_df["fashion_news_frequency"] = customer_df["fashion_news_frequency"].replace({"None":"NONE"})
customer_df["fashion_news_frequency"].fillna("UNKNOWN", inplace=True)

# Set missing values in age with the median
customer_df["age"].fillna(customer_df["age"].median(), inplace=True)
customer_df["age_interval"] = customer_df["age"].apply(lambda x: create_age_interval(x))

In [10]:
# Replace missing values
article_df.fillna(value="No Description", inplace=True)

# Adjust the article ID and product code to be string & add "0"
#article_df["product_code"] = article_df["article_id"].apply(lambda x: x[:3])

#article_df.head()

In [11]:
article_df['article_id'] = article_df['article_id'].astype(str)
article_df['article_id'] = article_df['article_id'].apply(lambda x: x.zfill(10))

unique_customer_ids = customer_df.customer_id.unique()
unique_article_ids = article_df.article_id.unique()

article_ds = tf.data.Dataset.from_tensor_slices(dict(article_df[['article_id']]))
articles = article_ds.map(lambda x: x['article_id'])

We subselect 1 years worth of data to use for all the exercises this week.

In [12]:
train0 = train0[train0['t_dat'] <='2019-09-20']

# add 0 in article_id column (string)
train0['article_id'] = train0['article_id'].astype(str)
train0['article_id'] = train0['article_id'].apply(lambda x: x.zfill(10))
train0.head()

<ipython-input-12-9af493a41d73>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train0['article_id'] = train0['article_id'].astype(str)
<ipython-input-12-9af493a41d73>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train0['article_id'] = train0['article_id'].apply(lambda x: x.zfill(10))


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [13]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

In [14]:
# Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)
def estimateRecall(cg, purchase):
    return len(intersection(cg,purchase))/len(purchase)

In [15]:
purchases = ["item45","item97","item71","item125","item5"]
cg1 = ["item1","item97","item12","item105","item5","item17","item197","item122","item85","item15"]
cg2 = ["item13","item94","item14","item15","item5","item18","item197","item132","item86","item65"]
print("recall for cg1: ",estimateRecall(cg1,purchases))
print("recall for cg1: ",estimateRecall(cg2,purchases))

recall for cg1:  0.4
recall for cg1:  0.2


# **Traditional Candidate Generation**

With the metric in place, now lets look into a few obvious candiate generators.

> **1. Candidate Genertator 1: random products**
The simplest candidate generator to start with is a random candidate selector -- from amongst the list of products, randomly select few products.


> **2. Candidate Genertator 2: Top most popular products**
Another simple candidate selection strategy is to select the top-k most popular products from historic interactions and use this list as the set of candidates for the downstream ranking goal.


> **3. Candidate Genertator 3: Two-tower model CG**
One of most popular, and industry-wide used CGs is a two-tower neural network. This is a learnt deep model with one tower for the user and another tower for the product, followed by a dot product. We describe this model in detail further down the notebook.


Below we will implement each of these three candidate generators and compare their performace in terms of recall metric.

In order to have a clean evaluation set-up, we will divide the transactions data we have into train and test split. We look at one year's of purchases and use the first 9 months as training data and the remaining three months as the test data.

In [16]:
#train0 = pd.read_csv('hmdata/transactions_train.csv.zip')
train = train0[train0['t_dat']<='2019-06-20']
test = train0[train0['t_dat'] >='2019-06-20']#[train0['t_dat'] <='2020-09-20']

For ease of evaluation, without any loss of generalization, we only consider customers that have made a purchase in both train and test time-periods. The methods and set-up remain the same for all other customers as well.

In [17]:
c1 = train['customer_id'].to_list()
c2 = test['customer_id'].to_list()
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

commonUsers = intersection(c1, c2)

print(len(train),len(test),len(c1),len(c2),len(commonUsers))

12241294 4686893 12241294 4686893 423753


Here commonUsers represent the set of users for whom we would be interested in generating candidates.

In [18]:
customer_id = transaction_df["customer_id"].iloc[0]

In [55]:
def getTopKCandidatesPopular(u,k):
    c = Counter(train['article_id'].to_list())
    topPopular = [article for article, count in c.most_common(k)]
    return topPopular

def get_recs_per_cust_based_on_other_users(customer_id):
    trans_df = train[train["customer_id"]==customer_id]
    cus_art_df = pd.merge(trans_df,article_df,on="article_id",how="inner")
    recommendations = []
    for index,sub_cust_art_df in cus_art_df.groupby(["index_name","section_name","department_name","product_type_name"]):
        cus_art_ids = sub_cust_art_df["article_id"].unique()
        other_trans_df = train[train["article_id"].isin(cus_art_ids)]
        others_same_buy_history = train[train["customer_id"].isin(other_trans_df["customer_id"].unique())]
        others_cus_article_df = pd.merge(others_same_buy_history,article_df,on="article_id")
        
        index_name = sub_cust_art_df["index_name"].unique()[0]
        section_name = sub_cust_art_df["section_name"].unique()[0]
        department_name = sub_cust_art_df["department_name"].unique()[0]
        product_type_name = sub_cust_art_df["product_type_name"].unique()[0]
        
        f1 = others_cus_article_df["index_name"]==index_name
        f2 = others_cus_article_df["section_name"]==section_name
        f3 = others_cus_article_df["department_name"]==department_name
        f4 = others_cus_article_df["product_type_name"]==product_type_name
        
        other_items = others_cus_article_df[f1 & f2 & f3 & f4]["article_id"].unique().tolist()
        recommendations.extend(other_items)
        return list(set(recommendations))
    
    
def get_recs_per_cust_based_on_popular_items(cusomer_id):
    trans_df = transaction_df[transaction_df["customer_id"]==customer_id]
    cus_art_df = pd.merge(trans_df,article_df,on="article_id",how="inner")
    recommendations = []
    pass
    

In [56]:
userSet = commonUsers[0:100]
overallRecall = 0
recallList = []
for u in userSet:
    purchaseList = test[test['customer_id']==u]['article_id'].to_list()
    cg1 = get_recs_per_cust_based_on_other_users(u)
#     cg2 = getTopKCandidatesPopular(u,100)
#     cg = cg1+cg2
    cg = cg1[:100]
    r = estimateRecall(cg,purchaseList)
    overallRecall+=r
    recallList.append(r)

overallRecall = overallRecall/len(userSet)

In [58]:
overallRecall

0.00973780487804878

In [57]:
recallList

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2222222222222222,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.1111111111111111,
 0.0,
 0.0,
 0.0,
 0.05,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.04878048780487805,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.041666666666666664,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [1]:
# for index,i in cus_art_df.iterrows():
#     index_name = i["index_name"]
#     section_name = i["section_name"]
#     department_name = i["department_name"]
#     product_type_name = i["product_type_name"]
    
#     f1 = others_cus_article_df["index_name"]==index_name
#     f2 = others_cus_article_df["section_name"]==section_name
#     f3 = others_cus_article_df["department_name"]==department_name
#     f4 = others_cus_article_df["product_type_name"]==product_type_name
    
#     merge_df = pd.merge(transaction_df,article_df,on="article_id",how="left")
#     break

    
